#

In [3]:
/*==============================================================================
   🏙 Spatial Convergence Analysis of NTL in India
   -------------------------------------------
   Purpose:
   This do-file explores regional convergence across Indian regions 
   between 1996 and 2010. It compares OLS and Spatial Durbin Model (SDM) 
   specifications (both unconditional and conditional), as well as models 
   with and without state fixed effects (FE). We use a spatial weights matrix 
   (WqueenS_fromStata15) to construct spatial lags.

   Key Features/Outcomes:
   1) OLS vs. SDM estimates with and without state FE.
   2) Extraction of AIC values for model comparison.
   3) Calculation of direct and indirect spatial impacts.
==============================================================================*/


/*------------------------------------------------------------------------------
   📚 1. BACKGROUND & CONTEXT
   ----------------------------------------
   - Research Motivation:
     Investigate how local characteristics and neighboring regions' values 
     affect light growth (a proxy for economic activity).

   - Theoretical Framework:
     Spatial Durbin Model (SDM) allows both the dependent variable and 
     explanatory variables to have spatial lags (i.e., influence from neighbors).

   - Data Description:
     1) "W_matrix.dta": Contains a 520×520 raw adjacency matrix or pre-constructed
        spatial weights.
     2) "india520.dta": Dataset with 520 observations representing regions/districts.
        Variables include:
         • light_growth96_10rcr_cap: Dependent variable (light-based growth).
         • log_light96_rcr_cap: Logged initial light measure.
         • Controls (terrain, climate, demographics, etc.): 
           suit_mean_snd, rain_mean_snd, mala_mean_snd, temp_mean_snd,
           rug_mean_snd, distance, latitude, rur_percent96_rcr,
           log_tot_density_rcr, sc_percent96, st_percent96, 
           workp_percent96, lit_percent96, higheredu_percent96, 
           elechh_percent96, log_puccaroads
         • state: Categorical variable for Indian states.

   - Key Variables:
     • light_growth96_10rcr_cap
     • log_light96_rcr_cap
     • WqueenS_fromStata15: Spatial weights matrix (row-normalized)
------------------------------------------------------------------------------*/


In [2]:
/*==============================================================================
   🌐 2. SETUP
   ======================================
   Purpose: 
   - Initialize Stata environment
   - Load required datasets
   - Define spatial settings and IDs
   - Prepare for regression analysis
==============================================================================*/

clear all
macro drop _all


* Basic environment settings
version 15
set more off


*------------------------------------------------------------------------------
* (2.1) Define a global macro for repeated control variables
*     Multi-line approach without quotes
*------------------------------------------------------------------------------
global controls suit_mean_snd rain_mean_snd ///
    mala_mean_snd temp_mean_snd ///
    rug_mean_snd distance ///
    latitude rur_percent96_rcr ///
    log_tot_density_rcr sc_percent96 ///
    st_percent96 workp_percent96 ///
    lit_percent96 higheredu_percent96 ///
    elechh_percent96 log_puccaroads

*------------------------------------------------------------------------------
* (2.2) Load Spatial Weights Matrix
*------------------------------------------------------------------------------
capture confirm file "W_matrix.dta"
if _rc != 0 {
    di as error "File W_matrix.dta not found. Please verify the file path."
    exit 1
}

use "W_matrix.dta", clear

* Generate ID for each observation and set spatial data
gen id = _n
order id, first
spset id

* Create a row-normalized spatial weights matrix from data
spmatrix fromdata WqueenS_fromStata15 = _0 - _519, normalize(row) replace

* Summarize the spatial weights to ensure correctness
spmatrix summarize WqueenS_fromStata15

*------------------------------------------------------------------------------
* (2.3) Load Main Dataset and Set Spatial IDs
*------------------------------------------------------------------------------
capture confirm file "india520.dta"
if _rc != 0 {
    di as error "File india520.dta not found. Please verify the file path."
    exit 1
}

use "india520.dta", clear
encode state, generate(state_encoded)
gen id = _n
spset id














      Sp dataset: W_matrix.dta
Linked shapefile: <none>
            Data: Cross sectional
 Spatial-unit ID: _ID (equal to id)
     Coordinates: <none>



Weighting matrix WqueenS_fromStata15
---------------------------------------
           Type |               custom
  Normalization |                  row
      Dimension |            520 x 520
Elements        |
   minimum      |                    0
   minimum > 0  |                   .1
   mean         |             .0019231
   max          |                    1
---------------------------------------







      Sp dataset: india520.dta
Linked shapefile: <none>
            Data: Cross sectional
 Spatial-unit ID: _ID (equal to id)
     Coordinates: <none>

In [4]:
/*==============================================================================
   🏗 3. MAIN ANALYSIS
   ======================================
   Purpose:
   - Estimate and compare multiple model specifications 
     (OLS vs. SDM, unconditional vs. conditional, with/without state FE).
   - Store AIC for model comparison.
   - Assess spatial impacts for SDM specifications.
==============================================================================*/

* We'll use eststo/estadd from the estout package to store results (e.g. AIC).

In [5]:
*------------------------------------------------------------------------------
* (3.1) Unconditional Models
*------------------------------------------------------------------------------

In [6]:
***************************************************************
* MODEL 1: Unconditional OLS
***************************************************************
regress light_growth96_10rcr_cap log_light96_rcr_cap, robust
eststo m1
estat ic
mat s = r(S)
quietly estadd scalar AIC = s[1,5]



Linear regression                               Number of obs     =        520
                                                F(1, 518)         =      94.27
                                                Prob > F          =     0.0000
                                                R-squared         =     0.2547
                                                Root MSE          =     .03722

-------------------------------------------------------------------------------
              |               Robust
light_growt~p | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
--------------+----------------------------------------------------------------
log_light96~p |  -.0198811   .0020476    -9.71   0.000    -.0239037   -.0158584
        _cons |  -.0722739   .0092154    -7.84   0.000     -.090378   -.0541698
-------------------------------------------------------------------------------



Akaike's information criterion and Bayesian information criterion

---------------

In [7]:
***************************************************************
* MODEL 2: Unconditional SDM
***************************************************************
quietly spregress light_growth96_10rcr_cap ///
    log_light96_rcr_cap, ///
    ml vce(robust) ///
    dvarlag(WqueenS_fromStata15) ///
    ivarlag(WqueenS_fromStata15: log_light96_rcr_cap)

eststo m2
estat ic
mat s = r(S)
quietly estadd scalar AIC = s[1,5]
estat impact





Akaike's information criterion and Bayesian information criterion

-----------------------------------------------------------------------------
       Model |          N   ll(null)  ll(model)      df        AIC        BIC
-------------+---------------------------------------------------------------
          m2 |        520          .   1144.248       5  -2278.497  -2257.228
-----------------------------------------------------------------------------
Note: BIC uses N = number of observations. See [R] IC note.




progress   :100% 

Average impacts                                 Number of obs     =        520

-------------------------------------------------------------------------------
              |            Delta-Method
              |      dy/dx   std. err.      z    P>|z|     [95% conf. interval]
--------------+----------------------------------------------------------------
direct        |
log_light96~p |  -.0196822   .0017798   -11.06   0.000    -.0231705   -.0161939


In [8]:
***************************************************************
* MODEL 3: Unconditional OLS with State FE
***************************************************************
regress light_growth96_10rcr_cap log_light96_rcr_cap i.state_encoded, robust
eststo m3
estat ic
mat s = r(S)
quietly estadd scalar AIC = s[1,5]



Linear regression                               Number of obs     =        520
                                                F(26, 491)        =          .
                                                Prob > F          =          .
                                                R-squared         =     0.7246
                                                Root MSE          =     .02324

-------------------------------------------------------------------------------
              |               Robust
light_growt~p | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
--------------+----------------------------------------------------------------
log_light96~p |  -.0216683   .0025087    -8.64   0.000    -.0265975   -.0167391
              |
state_encoded |
Arunachal ..  |   .0072529    .010514     0.69   0.491    -.0134051    .0279109
       Bihar  |   -.051117   .0084225    -6.07   0.000    -.0676656   -.0345684
  Chandigarh  |  -.0037725   .0027363    -1.38   0.16

In [9]:
***************************************************************
* MODEL 4: Unconditional SDM with State FE
***************************************************************
quietly spregress light_growth96_10rcr_cap ///
    log_light96_rcr_cap i.state_encoded, ///
    ml vce(robust) ///
    dvarlag(WqueenS_fromStata15) ///
    ivarlag(WqueenS_fromStata15: log_light96_rcr_cap i.state_encoded)

eststo m4
estat ic
mat s = r(S)
quietly estadd scalar AIC = s[1,5]
estat impact





Akaike's information criterion and Bayesian information criterion

-----------------------------------------------------------------------------
       Model |          N   ll(null)  ll(model)      df        AIC        BIC
-------------+---------------------------------------------------------------
          m4 |        520          .    1284.63      56   -2457.26  -2219.046
-----------------------------------------------------------------------------
Note: BIC uses N = number of observations. See [R] IC note.




progress   :  3%   7%  10%  14%  17%  21%  24%  28%  31%  34%  38%  41% 
              45%  48%  52%  55%  59%  62%  66%  69%  72%  76%  79%  83% 
              86%  90%  93%  97% 100% 

Average impacts                                 Number of obs     =        520

-------------------------------------------------------------------------------
              |            Delta-Method
              |      dy/dx   std. err.      z    P>|z|     [95% conf. interval]
--------

In [10]:
*------------------------------------------------------------------------------
* (3.2) Conditional Models
*     - Models that include the controls stored in $controls
*------------------------------------------------------------------------------

In [11]:
***************************************************************
* MODEL 5: Conditional OLS
***************************************************************
regress light_growth96_10rcr_cap ///
    log_light96_rcr_cap ///
    $controls, robust

eststo m5
estat ic
mat s = r(S)
quietly estadd scalar AIC = s[1,5]



Linear regression                               Number of obs     =        520
                                                F(17, 502)        =      26.59
                                                Prob > F          =     0.0000
                                                R-squared         =     0.5797
                                                Root MSE          =     .02839

-------------------------------------------------------------------------------
              |               Robust
light_growt~p | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
--------------+----------------------------------------------------------------
log_light96~p |  -.0246757   .0031661    -7.79   0.000    -.0308961   -.0184554
suit_mean_snd |  -.0311005   .0069184    -4.50   0.000    -.0446932   -.0175078
rain_mean_snd |  -1.73e-06   2.07e-06    -0.84   0.402    -5.80e-06    2.33e-06
mala_mean_snd |    .001687   .0062859     0.27   0.789    -.0106629    .0140368
temp_

In [12]:
***************************************************************
* MODEL 6: Conditional SDM
***************************************************************
quietly spregress light_growth96_10rcr_cap ///
    log_light96_rcr_cap ///
    $controls, ///
    ml vce(robust) ///
    dvarlag(WqueenS_fromStata15) ///
    ivarlag(WqueenS_fromStata15: log_light96_rcr_cap $controls)

eststo m6
estat ic
mat s = r(S)
quietly estadd scalar AIC = s[1,5]
estat impact





Akaike's information criterion and Bayesian information criterion

-----------------------------------------------------------------------------
       Model |          N   ll(null)  ll(model)      df        AIC        BIC
-------------+---------------------------------------------------------------
          m6 |        520          .   1209.706      37  -2345.411   -2188.02
-----------------------------------------------------------------------------
Note: BIC uses N = number of observations. See [R] IC note.




progress   :  6%  12%  18%  24%  29%  35%  41%  47%  53%  59%  65%  71% 
              76%  82%  88%  94% 100% 

Average impacts                                 Number of obs     =        520

-------------------------------------------------------------------------------
              |            Delta-Method
              |      dy/dx   std. err.      z    P>|z|     [95% conf. interval]
--------------+----------------------------------------------------------------
di

In [13]:
***************************************************************
* MODEL 7: Conditional OLS with State FE
***************************************************************
regress light_growth96_10rcr_cap ///
    log_light96_rcr_cap ///
    $controls ///
    i.state_encoded, robust

eststo m7
estat ic
mat s = r(S)
quietly estadd scalar AIC = s[1,5]



Linear regression                               Number of obs     =        520
                                                F(42, 475)        =          .
                                                Prob > F          =          .
                                                R-squared         =     0.7677
                                                Root MSE          =      .0217

-------------------------------------------------------------------------------
              |               Robust
light_growt~p | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
--------------+----------------------------------------------------------------
log_light96~p |  -.0246248   .0028208    -8.73   0.000    -.0301675   -.0190821
suit_mean_snd |  -.0197893   .0078754    -2.51   0.012    -.0352643   -.0043143
rain_mean_snd |  -5.63e-07   1.96e-06    -0.29   0.774    -4.42e-06    3.29e-06
mala_mean_snd |  -.0065129   .0056121    -1.16   0.246    -.0175405    .0045147
temp_

In [14]:
***************************************************************
* MODEL 8: Conditional SDM with State FE
***************************************************************
quietly spregress light_growth96_10rcr_cap ///
    log_light96_rcr_cap ///
    $controls ///
    i.state_encoded, ///
    ml vce(robust) ///
    dvarlag(WqueenS_fromStata15) ///
    ivarlag(WqueenS_fromStata15: ///
        log_light96_rcr_cap ///
        $controls ///
        i.state_encoded)

eststo m8
estat ic
mat s = r(S)
quietly estadd scalar AIC = s[1,5]
estat impact






Akaike's information criterion and Bayesian information criterion

-----------------------------------------------------------------------------
       Model |          N   ll(null)  ll(model)      df        AIC        BIC
-------------+---------------------------------------------------------------
          m8 |        520          .   1319.159      88  -2462.319  -2087.982
-----------------------------------------------------------------------------
Note: BIC uses N = number of observations. See [R] IC note.




progress   :  2%   4%   7%   9%  11%  13%  16%  18%  20%  22%  24%  27% 
              29%  31%  33%  36%  38%  40%  42%  44%  47%  49%  51%  53% 
              56%  58%  60%  62%  64%  67%  69%  71%  73%  76%  78%  80% 
              82%  84%  87%  89%  91%  93%  96%  98% 100% 

Average impacts                                 Number of obs     =        520

-------------------------------------------------------------------------------
              |            Delta-M

    \begin{table}[htbp]
    \centering
    \caption{Unconditional and conditional convergence across districts}
    \label{tab:combined_convergence}
    \resizebox{\textwidth}{!}{%
    \begin{tabular}{@{}lcccccccc@{}}
    \toprule
    & \multicolumn{4}{c}{Unconditional} & \multicolumn{4}{c}{Conditional} \\
    \cmidrule(lr){2-5} \cmidrule(l){6-9}
    Effect & OLS & SDM & OLS & SDM & OLS & SDM & OLS & SDM \\ \midrule
    Direct & -0.020*** & -0.020*** & -0.022*** & -0.021*** & -0.025*** & -0.026*** & -0.025*** & -0.025*** \\
     & (0.002) & (0.002) & (0.003) & (0.002) & (0.003) & (0.002) & (0.003) & (0.002) \\
    Indirect & -- & -0.002 & -- & -0.008 & -- & -0.008 & -- & -0.009* \\
     & & (0.004) & & (0.006) & & (0.006) & & (0.005) \\
    Total & -0.020*** & -0.022*** & -0.022*** & -0.029*** & -0.025*** & -0.034*** & -0.025*** & -0.034*** \\
     & (0.002) & (0.005) & (0.003) & (0.007) & (0.003) & (0.007) & (0.003) & (0.005) \\ \midrule
    Controls & No & No & No & No & Yes & Yes & Yes & Yes \\
    State FE & No & No & Yes & Yes & No & No & Yes & Yes \\ 
    AIC & -1945 & -2278 & -2413 & -2457 & -2211 & -2345 & -2469 & -2462 \\ \bottomrule
    \end{tabular}%
    }
    \footnotesize
    \begin{tablenotes}
    \item \textit{Note}: Standard errors in parentheses. Significance: ***p$<$0.01, **p$<$0.05, *p$<$0.1.
    \end{tablenotes}
    \end{table}

**Unconditional and conditional convergence across districts**

| Effect | OLS | SDM | OLS | SDM | OLS | SDM | OLS | SDM |
|----|---------|---------|---------|---------|---------|---------|---------|---------|
| Direct | -0.020\*\*\*<br>(0.002) | -0.020\*\*\*<br>(0.002) | -0.022\*\*\*<br>(0.003) | -0.021\*\*\*<br>(0.002) | -0.025\*\*\*<br>(0.003) | -0.026\*\*\*<br>(0.002) | -0.025\*\*\*<br>(0.003) | -0.025\*\*\*<br>(0.002) |
| Indirect | – | -0.002<br>(0.004) | – | -0.008<br>(0.006) | – | -0.008<br>(0.006) | – | -0.009\*<br>(0.005) |
| Total | -0.020\*\*\*<br>(0.002) | -0.022\*\*\*<br>(0.005) | -0.022\*\*\*<br>(0.003) | -0.029\*\*\*<br>(0.007) | -0.025\*\*\*<br>(0.003) | -0.034\*\*\*<br>(0.007) | -0.025\*\*\*<br>(0.003) | -0.034\*\*\*<br>(0.005) |
| Controls | No | No | No | No | Yes | Yes | Yes | Yes |
| State FE | No | No | Yes | Yes | No | No | Yes | Yes |
| AIC | -1945 | -2278 | -2413 | -2457 | -2211 | -2345 | -2469 | -2462 |

*Note: Standard errors in parentheses. Significance: ***p\<0.01,
**p\<0.05, *p\<0.1.*